In [1]:
import math
import numpy as np
from typing import List, Union
from scipy import special

In [6]:
# Setting: constant noise
client_rate = 1.0
sample_rate = 0.1
rounds = 100
steps = 2
delta = 0.00001
delta_g = 0.00021
eta = 0.5
noise_multiplier = 5.0
k = rounds * steps

In [6]:
# one DP
eps_dp = math.sqrt(2 * math.log(1.25 / delta)) / noise_multiplier
eps_dp = np.log(1 + sample_rate * (np.exp(eps_dp) - 1))
delta_dp = delta * sample_rate
print(f"DP privacy cost of one iteration: {eps_dp}, delta: {delta_dp}")

DP privacy cost of one iteration: 0.1514503397740035, delta: 1.0000000000000002e-06


In [10]:
# one RDP
def generate_rdp_orders():
    dense = 1.07
    alpha_list = [int(dense ** i + 1) for i in range(int(math.floor(math.log(1000, dense))) + 1)]
    alpha_list = np.unique(alpha_list)
    return alpha_list

def _log_add(logx: float, logy: float) -> float:
    a, b = min(logx, logy), max(logx, logy)
    if a == -np.inf:  # adding 0
        return b
    # Use exp(a) + exp(b) = (exp(a - b) + 1) * exp(b)
    return math.log1p(math.exp(a - b)) + b  # log1p(x) = log(x + 1)

def _compute_log_a_for_int_alpha(q: float, sigma: float, alpha: int) -> float:
    # Initialize with 0 in the log space.
    log_a = -np.inf

    for i in range(alpha + 1):
        log_coef_i = (
            math.log(special.binom(alpha, i))
            + i * math.log(q)
            + (alpha - i) * math.log(1 - q)
        )
        if i >= 3:
            log_coef_i = log_coef_i + math.log(3)

        s = log_coef_i + (i * i - i) / (2 * (sigma ** 2))

        log_a = _log_add(log_a, s)

    return float(log_a)

def _compute_log_a(q: float, sigma: float, alpha: float) -> float:
    return _compute_log_a_for_int_alpha(q, sigma, int(alpha))

def _compute_rdp(q: float, sigma: float, alpha: float) -> float:
    if q == 0:
        return 0

    # no privacy
    if sigma == 0:
        return np.inf

    if q == 1.0:
        return alpha / (2 * sigma**2)

    if np.isinf(alpha):
        return np.inf

    return _compute_log_a(q, sigma, alpha) / (alpha - 1)

def compute_rdp(
    *, q: float, noise_multiplier: float, steps: int, orders: Union[List[float], float]
) -> Union[List[float], float]:
    if isinstance(orders, float):
        rdp = _compute_rdp(q, noise_multiplier, orders)
    else:
        rdp = np.array([_compute_rdp(q, noise_multiplier, order) for order in orders])

    return rdp * steps

# compute the privacy cost of a step
def compute_privacy_cost_one_step(noise_multiplier, sample_rate, delta):
    alphas = generate_rdp_orders()

    inner_rdp = compute_rdp(q=sample_rate, noise_multiplier=noise_multiplier, steps=1,
                                         orders=alphas)
    orders_vec = np.atleast_1d(alphas)
    rdp_vec = np.atleast_1d(inner_rdp)
    eps_vec = (
        rdp_vec
        - (np.log(delta) + np.log(orders_vec)) / (orders_vec - 1)
        + np.log((orders_vec - 1) / orders_vec)
    )
    idx_opt = np.nanargmin(eps_vec)
    eps = eps_vec[idx_opt]

    return eps, delta

eps_rdp, delta_rdp = compute_privacy_cost_one_step(
    noise_multiplier=noise_multiplier, sample_rate=sample_rate, delta=delta_g
)
print(f"RDP privacy cost of one iteration: {eps_rdp}, delta: {delta_rdp}")

RDP privacy cost of one iteration: 0.07582816727040625, delta: 0.00021


In [13]:
# Basic composition method
eps1 = eps_dp * k
delta1 = k * delta_dp
print(f"Basic composition method: {eps1}, delta: {delta1}")

Basic composition method: 30.290067954800698, delta: 0.00020000000000000004


In [14]:
# Advanced composition method
delta_prime = 0.00001
eps2 = math.sqrt(2 * k * math.log(1 / delta_prime)) * eps_dp + k * eps_dp * (np.exp(eps_dp) - 1)
delta2 = k * delta_dp + delta_prime
print(f"Advanced composition method: {eps2}, delta: {delta2}")

Advanced composition method: 15.230680383519832, delta: 0.00021000000000000004


In [16]:
# RDP composition method
def get_pc_rdp(
        sample_rate,
        rounds,
        steps,
        noise_multiplier,
        delta
):
    eps, delta = compute_privacy_cost_one_step(
        noise_multiplier=noise_multiplier, sample_rate=sample_rate, delta=delta
    )
    k = rounds * steps
    eps_prime = 4 * eps * math.sqrt(2 * k * np.log(1 / delta))

    return eps_prime, delta

eps3, delta3 = get_pc_rdp(sample_rate, rounds, steps, noise_multiplier, delta=delta_g)
print(f"RDP composition method: {eps3}, delta: {delta3}")

RDP composition method: 17.653113186532675, delta: 0.00021


In [7]:
# DIARY's privacy cost estimation (PCE)
def F_dp(r, s, weights, values):
    # Initialize a DP table with (r+1) rows and (s+1) columns
    dp = [[0] * (s + 1) for _ in range(r + 1)]

    # Base case: when no items are selected, the value is 1
    for j in range(s + 1):
        dp[0][j] = 1

    # Fill the DP table
    for i in range(1, r + 1):
        ar = weights[i - 1]
        wr = values[i - 1]
        for j in range(s + 1):
            if ar <= j:
                dp[i][j] = dp[i - 1][j] + wr * dp[i - 1][j - ar]
            else:
                dp[i][j] = dp[i - 1][j]

    return dp[r][s]

def calculate_optcomp(k, epsilons, deltas, a_g, eps_0, delta_g, a_i):
    epsilon_g = a_g * eps_0

    # compute left_side_result
    # Calculate the product (1 + exp(epsilon_i))
    product_term = np.prod([1 + np.exp(e) for e in epsilons])

    # compute F(k,B)
    B = (np.sum(a_i) - a_g) // 2

    values_1 = [np.exp(-e) for e in epsilons]
    F_1 = F_dp(k, B, a_i, values_1)

    values_2 = [np.exp(e) for e in epsilons]
    F_2 = F_dp(k, B, a_i, values_2)

    sum_term = np.prod([np.exp(e) for e in epsilons]) * F_1 - np.exp(epsilon_g) * F_2

    # Divide by product term
    left_side_result = sum_term / product_term

    # compute right_side_result
    # Calculate the product (1 - delta)
    product_term_ = np.prod([1 - d for d in deltas])

    right_side_result = 1 - ((1 - delta_g) / product_term_)

    return left_side_result - right_side_result

def binary_search_epsilon_g(eps_0, k, epsilons, deltas, delta_g, a_i):
    a, b = 1, sum(a_i)  # Initial bounds
    i = 0
    while b >= a:
        i += 1
        m = (a + b) // 2
        result = calculate_optcomp(k, epsilons, deltas, m, eps_0, delta_g, a_i)
        if result < 0:
            b = m - 1
        elif result > 0:
            a = m + 1
        else:
            return m, m * eps_0
    return ((a + b) // 2 + 1), ((a + b) // 2 + 1) * eps_0

def get_privacy_spent(privacy_costs, deltas, delta_g, eta):
    """compute the global privacy loss"""
    eps_mean = sum(privacy_costs) / len(privacy_costs)
    beta = eta / (len(privacy_costs) * (1 + eps_mean) + 1)
    eps_0 = np.log(1 + beta)
    a = []
    eps_pie = []
    for eps_i in privacy_costs:
        a_i = math.ceil(eps_i * (1 / beta + 1))
        eps_i_pie = eps_0 * a_i
        a.append(a_i)
        eps_pie.append(eps_i_pie)
    a_g, epsilon_g = binary_search_epsilon_g(eps_0, len(privacy_costs), eps_pie,
                                             deltas, delta_g, a)
    return a_g, epsilon_g

def pce(sample_rate,
        rounds,
        steps,
        noise_multiplier,
        delta,
        delta_g,
        eta
):
    privacy_costs = []
    deltas = []
    eps = math.sqrt(2 * math.log(1.25 / delta)) / noise_multiplier
    eps = np.log(1 + sample_rate * (np.exp(eps) - 1))
    delta = delta * sample_rate
    for i in range(int(rounds * steps)):
        privacy_costs.append(eps)
        deltas.append(delta)
    a_t, epsilon_t = get_privacy_spent(privacy_costs, deltas, delta_g=delta_g, eta=eta)
    return epsilon_t, a_t

eps4, a_1 = pce(sample_rate, rounds, steps, noise_multiplier, delta, delta_g, eta)
print(f"DIARY's PCE: {eps4}, a_g: {a_1}, delta_g: {delta_g}")

DIARY's PCE: 10.909563100351484, a_g: 5052, delta_g: 0.00021
